In [ ]:
#|default_exp metadata

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#|export

from collections import defaultdict
from datetime import datetime
from math import ceil
from pathlib import Path

import numpy as np

from clip_plot.configuration import ClusterSpec, UmapSpec
from clip_plot.hotspots import get_hotspots
from clip_plot.layouts import get_heightmap, get_layouts
from clip_plot.utils import FILE_NAME, clean_filename, get_version, is_number, read_json, write_json


# Handle metadata (dates, categories, etc.)

In [ ]:
#| export

def write_metadata(imageEngine):
    """
    Write list `metadata` of objects to disk
    Tightly coupled to ImageEngine class
    """
    metadata = imageEngine.metadata
    data_dir = imageEngine.data_dir
    if not metadata:
        return

    meta_dir = data_dir / "metadata"
    for subdir in ["filters", "options", "file"]:
        (meta_dir / subdir).mkdir(parents=True, exist_ok=True)

    # Create the lists of images with each tag
    d = defaultdict(list)
    for img in imageEngine:
        row = img.metadata
        filename = img.unique_name
        row["tags"] = [j.strip() for j in row.get("tags", "").split("|")]
        for j in row["tags"]:
            d["__".join(j.split())].append(filename)
        write_json((meta_dir/"file"/f"{filename}.json"),
                    data_dir=data_dir, obj=row)

    write_json(output_path=(meta_dir/"filters"/"filters.json"),
               data_dir=data_dir,
               obj=[{"filter_name": "select",
               "filter_values": list(d.keys()),
        }],
    )

    # create the options for the category dropdown
    for i in d:
        write_json(output_path=meta_dir/f"options/{i}.json",
                   data_dir=data_dir, obj=d[i])

    # create the map from date to images with that date (if dates present)
    date_d = defaultdict(list)
    for i in metadata:
        date = i.get("year", "")
        if date:
            date_d[date].append(clean_filename(i[FILE_NAME]))

    # find the min and max dates to show on the date slider
    dates = np.array([int(i.strip()) for i in date_d if is_number(i)])
    domain = {"min": float("inf"), "max": -float("inf")}
    mean = np.mean(dates)
    std = np.std(dates)
    for i in dates:
        # update the date domain with all non-outlier dates
        if abs(mean - i) < (std * 4):
            domain["min"] = int(min(i, domain["min"]))
            domain["max"] = int(max(i, domain["max"]))

    # write the dates json
    if len(date_d) > 1:
        write_json(output_path=meta_dir/"dates.json",
                   data_dir=data_dir,
                   obj={"domain": domain, "dates": date_d},
        )

In [ ]:
#| export

def get_manifest(imageEngine, atlas_data,
                 hidden_vectors: np.ndarray,
                 plot_id: str,
                 output_dir: Path,
                 umap_spec: UmapSpec,
                 cluster_spec: ClusterSpec,
                 has_metadata: bool = False,
                 x_col: str | None = None,
                 y_col: str | None = None,
                 ):
    """
    Create and return the base object for the manifest output file
    All paths are relative to data_dir
    """
    data_dir = output_dir/"data"
    # store each cell's size and atlas position
    atlas_ids = {i["idx"] for i in atlas_data}
    sizes = [[] for _ in atlas_ids]
    pos = [[] for _ in atlas_ids]
    for i in atlas_data:
        sizes[i["idx"]].append([i["w"], i["h"]])
        pos[i["idx"]].append([i["x"], i["y"]])

    # obtain the paths to each layout's JSON positions
    layouts = get_layouts(imageEngine, hidden_vectors,
                                   data_dir, plot_id, umap_spec,
                                   x_col, y_col)
    # create a heightmap for the umap layout
    if "umap" in layouts and layouts["umap"]:
        get_heightmap(json_path=layouts["umap"]["variants"][0]["layout"],
                      label="umap", data_dir=data_dir)

    # specify point size scalars
    point_sizes = {}
    point_sizes["min"] = 0
    point_sizes["grid"] = 1 / ceil(imageEngine.count ** (1 / 2))
    point_sizes["max"] = point_sizes["grid"] * 1.2
    point_sizes["scatter"] = point_sizes["grid"] * 0.2
    point_sizes["initial"] = point_sizes["scatter"]
    point_sizes["categorical"] = point_sizes["grid"] * 0.6
    point_sizes["geographic"] = point_sizes["grid"] * 0.025

    # fetch the date distribution data for point sizing
    if "date" in layouts and layouts["date"]:
        date_layout = read_json(layouts["date"]["labels"])
        point_sizes["date"] = 1 / (
            (date_layout["cols"] + 1) * len(date_layout["labels"])
        )

    # create manifest json
    manifest = {
        "version": get_version(),
        "plot_id": plot_id,
        "output_directory": ".",
        "layouts": layouts,
        "initial_layout": "umap",
        "point_sizes": point_sizes,
        "imagelist": "data/imagelist.json",
        "atlas_dir": f"data/atlases/{plot_id}",
        "metadata": has_metadata,
        "default_hotspots": get_hotspots(imageEngine, hidden_vectors,
                                         data_dir, plot_id,
                                         cluster_spec=cluster_spec,
                                        #  layouts=layouts,
                                         ),
        "custom_hotspots": None,
        "gzipped": False,
        "config": {
            "sizes": {
                "atlas": imageEngine.atlas_size,
                "cell": imageEngine.atlas_row_height,
                "lod": imageEngine.thumbnail_size,
            },
        },
        "creation_date": datetime.today().strftime("%d-%B-%Y-%H:%M:%S"),
    }

    write_json(data_dir/"manifests/manifest.json", data_dir=data_dir, obj=manifest)
    write_json(data_dir/"manifest.json", data_dir=data_dir, obj=manifest)

    imagelist = {
        "cell_sizes": sizes,
        "images": [img.unique_name for img in imageEngine],
        "atlas": {
            "count": len(atlas_ids),
            "positions": pos,
        },
    }

    write_json(data_dir / "imagelist.json",
               data_dir=data_dir, obj=imagelist)

In [ ]:
#|hide

import nbdev

nbdev.nbdev_export()